# <u><p style="text-align: center;">Dataframes</p></u>

### Learning goals  
Students will:  
* Learn about Spark Dataframes

### Background

As we have seen, RDDs are the building blocks of Spark. RDDs have several advantages but in some cases their use can be problematic. Such cases can occur because Spark does not optimize transformations when we perform them directly to RDDs. Another example is that working with RDDs in some programming languages (including Python) can lead to poor performance. Also, transformation chains with RDDs can be difficult to comprehend since they show how the result will be achieved but not what the result will be.

Spark **DataFrames** were conceived to overcome the aforementioned problems. Similar to RDDs, DataFrames are distributed collections of data. The difference is that DataFrames provide a high-level abstraction over RDDs that allows us to use a query language to manipulate data. This abstraction is a logical plan that represents data and a schema. The logical plan is converted to a physical plan for execution. This conversion brings us closer to **what** we want to do rather than **how** we have to do it, because we let Spark figure out the most efficient way to carry out the operations. Dataframes are generally faster than RDDs, and they perform the same no matter what programming language we use with Spark.

### Code examples

Before proceeding to the examples, we are going to initialize Spark:

In [ ]:
from pyspark import SparkContext
import os

#'swan_spark_conf' is a configuration provided by a plugin for Jupyter. We further extend this configuration with proxy settings.
swan_spark_conf = swan_spark_conf.setAll([('spark.ui.proxyBase', os.environ['JUPYTERHUB_SERVICE_PREFIX'] + 'proxy/4040')])

#instantiate a SparkContext object with our configuration
sc = SparkContext.getOrCreate(conf=swan_spark_conf)

#### Example 1:

DataFram

#### Example 2:

DataFrames provide a convenient way to work with tabular data. In this example, we are going to read a file with Spark and convert into a DataFrame. The file contains the minimum and maximum daily temperatures for the years 2010-2015 in De Bilt, Netherlands. 

Then, we are going to find the minimum and maximum temperatures that occured during these years, the date when they happened, and also count how many days the temperature was below 0 $^\text{o}C$.

So, the first step is to load the data into a DataFrame:

In [ ]:
dataDF = sc.read.csv('de_bilt.csv', header=True, inferSchema=True)

and then we examine how the data look like by using the function `show`:

In [ ]:
dataDF.show()

Dates are formatted as YYYYMMDD, temperatures are in Celcius degrees.

Next, to find the minimum and maximum temperatures we are going to use **aggregations** over the DataFrame. We can perform aggregations by using the `agg` function. The parameters of `agg` are expressions that indicate the aggregation that we want to perform. To find the maximum temperature a possible solution is:

In [ ]:
from pyspark.sql import functions as F

dataDF.agg(F.max("Tmax")).show() #notice that Tmax is the name of the column

and similarly for the minimum:

In [ ]:
dataDF.agg(F.min("Tmin")).show() #notice that Tmin is the name of the column

Now, to find how many days the temperature was below 0 $^\text{o}C$ we are going to use the `filter` function with an appropriate conditional:

In [ ]:
below_zeroDF = dataDF.filter(F.col("Tmin") < 0)
below_zeroDF.count()

#### Example 2:

### Quiz

### Further reading

* 